In [1]:
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 8.6 MB/s eta 0:00:00


In [57]:
# Begin with the general recursive Solovay-Kitaev framework as given in Dawson & Nielsen 2005

import numpy as np
import math



def inverse_sequence(seq):
  out = ''.join(seq.swapcase() for char in seq)
  return out[::-1]

def dagger(A: np.ndarray):
    return A.conj().T

# Return the sequence of gates to construct the unitary + the actual approximation matrix itself
def solovay_kitaev(U,n): # input :: gate , depth
  if(n==0): # base case
    return enet_approx(U) # returns the sequence
  U_n_1_seq, U_n_1 = solovay_kitaev(U, n-1)
  #U_prev = multiply(U_prev_seq)
  print(U @ dagger(U_n_1))
  V,W = GC_decompose(U @ dagger(U_n_1))
  V_n_1_seq, V_n_1 = solovay_kitaev(V,n-1)
  W_n_1_seq, W_n_1 = solovay_kitaev(W,n- 1)
  V_inv_seq = inverse_sequence(V_n_1_seq)

  W_inv_seq = inverse_sequence(W_n_1_seq)


  U_n = V_n_1 @ W_n_1 @ dagger(V_n_1) @ dagger(U_n_1) @ U_n_1
  U_n_seq = V_n_1_seq + W_n_1_seq + inverse_sequence(V_n_1_seq) + inverse_sequence(W_n_1_seq) + U_n_1_seq
  return U_n_seq, U_n

In [46]:
# Building an initial enet
enet = []
enet_strs = []

def build_enet(gateset, max_depth, gateset_desc):

  global enet, enet_strs
  if max_depth <= 0:
    return
  if len(enet) == 0:
    enet = list(gateset.copy())
    enet_strs = list(gateset_desc.copy())
    build_enet(gateset, max_depth - 1, gateset_desc)
  else:
    # Take every element of the enet and left multiply it by every element in the gateset
    for i in range(len(gateset)):
      gate = gateset[i]

      current = len(enet)
      for j in range(current):

        thing = gate @ enet[j]
        if np.any(np.all(enet == thing, axis = (1, 2))):
          continue

        enet.append(gate @ enet[j])
        enet_strs.append(gateset_desc[i] + enet_strs[j])

    build_enet(gateset, max_depth - 1, gateset_desc)

def enet_approx(U):
  trace_norms = np.linalg.norm(enet - U, ord='nuc', axis=(1, 2))

  # Find the index of the matrix in the array closest in trace norm to the given matrix
  closest_index = np.argmin(np.abs(trace_norms))

  return enet[closest_index]


# This is a rough approximation for an actual enet that just brute forces through all gateset words up to depth max_depth
build_enet(np.array((np.array([[1, 2], [0, 1]]), np.array([[1, 2], [2, 2]]))), 5, ['a', 'b'])
enet = np.array(enet)
print(enet, enet_strs)
print(enet_approx(np.array([[1, 0], [0, 1]])))



[[[   1    2]
  [   0    1]]

 [[   1    2]
  [   2    2]]

 [[   1    4]
  [   0    1]]

 [[   5    6]
  [   2    2]]

 [[   1    4]
  [   2    6]]

 [[   5    6]
  [   6    8]]

 [[   1    6]
  [   2   10]]

 [[   9   10]
  [  14   16]]

 [[   1    6]
  [   0    1]]

 [[   9   10]
  [   2    2]]

 [[   5   16]
  [   2    6]]

 [[  17   22]
  [   6    8]]

 [[   5   26]
  [   2   10]]

 [[  37   42]
  [  14   16]]

 [[   5   16]
  [   6   20]]

 [[  17   22]
  [  22   28]]

 [[   5   26]
  [   6   32]]

 [[  37   42]
  [  46   52]]

 [[   1    8]
  [   2   14]]

 [[  13   14]
  [  22   24]]

 [[   9   28]
  [  14   44]]

 [[  29   38]
  [  46   60]]

 [[   9   46]
  [  14   72]]

 [[  65   74]
  [ 102  116]]

 [[   1    8]
  [   0    1]]

 [[  13   14]
  [   2    2]]

 [[   9   28]
  [   2    6]]

 [[  29   38]
  [   6    8]]

 [[   9   46]
  [   2   10]]

 [[  65   74]
  [  14   16]]

 [[  17   56]
  [   6   20]]

 [[  61   78]
  [  22   28]]

 [[  17   90]
  [   6   32]]

 [[ 129  1

In [36]:
# Turn arbitrary unitary matrix into special unitary by normalizing determinant
def SU2(U):
  determinant = np.linalg.det(U)
  return (1/determinant)**0.5 * U

def Rx(phi):
  return np.array([[np.cos(phi/2), -1j*np.sin(phi/2)], [-1j*np.sin(phi/2), np.cos(phi/2)]])

def Ry(phi):
  return np.array([[np.cos(phi/2), -np.sin(phi/2)], [np.sin(phi/2), np.cos(phi/2)]])


def GC_decompose(U):
  theta = 2*math.acos(U[0][0].real)

  phi = 2*math.asin(math.sqrt(math.sqrt(0.5*(1+math.sqrt(1 - math.sin(theta/2.0)**2)))))
  V = SU2(Rx(phi))
  W = SU2(Ry(phi))
  Winv = dagger(W)
  Vinv = dagger(V)
  comm = V @ W @ Vinv @ Winv
  w1,v1= np.linalg.eig(U)
  w2,v2= np.linalg.eig(comm)

  S = SU2(v1 @ dagger(v2))

  Vout = S @ V @ dagger(S)
  Wout = S @ W @ dagger(S)
  return Vout, Wout

In [40]:
# Test
V, W = GC_decompose([[0, 1], [1, 0]])
print(V @ W @ dagger(V) @ dagger(W))  # phase invariant?, idk

[[-1.11022302e-16-3.36612407e-16j  5.55111512e-17-1.00000000e+00j]
 [ 5.55111512e-17-1.00000000e+00j -3.88578059e-16-1.14992425e-16j]]


In [58]:
# Test Solovay-Kitaev
print(solovay_kitaev(np.array([[1, 0], [0, 1]]), 5))

[0 1]


IndexError: invalid index to scalar variable.